In [2]:
from galpy.orbit import Orbit
from galpy.potential import MWPotential2014
from numpy import *

o= Orbit.from_name('Omega Cen')
R = 8.0
vR = 0.0
vT = 220.0
z = 0.0
vz = 0.0
phi = 0.0
o= Orbit(vxvv=[R,vR,vT,z,vz,phi])
ts= numpy.linspace(0.,100.,2001)
o.integrate(ts,MWPotential2014)
o.plot()
plot([o.R()],[o.z()],'ro')

/Users/ciaranohare/anaconda/lib/python2.7/site-packages/cryptography/hazmat/primitives/constant_time.py:26: CryptographyDeprecationWarning: Support for your Python version is deprecated. The next version of cryptography will remove support. Please upgrade to a 2.7.x release that supports hmac.compare_digest as soon as possible.
  utils.DeprecatedIn23,


NameError: name 'numpy' is not defined